In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_largest_companies_in_the_United_States_by_revenue'

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

page = requests.get(url, headers=headers)

soup = BeautifulSoup(page.text, "html.parser")

In [4]:
table = soup.find("table", {"class": "wikitable sortable"})

In [5]:
headers = [th.text.strip() for th in table.find_all("th")]

In [6]:
rows = []
for tr in table.find_all("tr")[1:]:
    tds = tr.find_all("td")
    if len(tds) == len(headers):
        rows.append([td.text.strip() for td in tds])

In [7]:
df_wiki = pd.DataFrame(rows, columns=headers)

In [8]:
len(df_wiki)

100

In [9]:
df_wiki

,Rank,Name,Industry,Revenue (USD millions),Revenue growth,Employees,Headquarters
0,1,Walmart,Retail,"680,985",5.1%,"2,100,000","Bentonville, Arkansas"
1,2,Amazon,Retail and cloud computing,"637,959",11.0%,"1,556,000","Seattle, Washington"
2,3,UnitedHealth Group,Healthcare,"400,278",7.7%,"400,000","Minnetonka, Minnesota"
3,4,Apple,Electronics industry,"391,035",2.0%,"164,000","Cupertino, California"
4,5,CVS Health,Healthcare,"372,809",4.2%,"259,500","Woonsocket, Rhode Island"
...,...,...,...,...,...,...,...
95,96,General Dynamics,Aerospace and defense,"47,716",12.9%,"117,000","Reston, Virginia"
96,97,Coca-Cola,Beverage,"47,061",2.9%,"69,700","Atlanta, Georgia"
97,98,TIAA,Financials,"46,946",2.6%,"15,623","New York City, New York"
98,99,The Travelers Companies,Insurance,"46,423",12.2%,"34,000","New York City, New York"


In [10]:
df_wiki = df_wiki.rename(columns={'Name': 'Company'})



In [11]:
df_wiki['Company'] = df_wiki['Company'].str.strip()


In [12]:
companies = df_wiki["Company"].tolist()
len(companies)


100

In [13]:
print(companies[:10])


['Walmart', 'Amazon', 'UnitedHealth Group', 'Apple', 'CVS Health', 'Berkshire Hathaway', 'Alphabet', 'ExxonMobil', 'McKesson Corporation', 'Cencora']


In [14]:
#API for matching name of company with their symbol
import finnhub
api_key = "d3t3huhr01qqdgfu1rngd3t3huhr01qqdgfu1ro0"
finnhub_client = finnhub.Client(api_key=api_key)


In [15]:
#Testing of company name in Finhub database
#We find 11 matches for company with "Apple" in thier name
result = finnhub_client.symbol_lookup("Apple")
print(result)


{'count': 11, 'result': [{'description': 'Apple Inc', 'displaySymbol': 'AAPL', 'symbol': 'AAPL', 'type': 'Common Stock'}, {'description': 'Apple Hospitality REIT Inc', 'displaySymbol': 'APLE', 'symbol': 'APLE', 'type': 'Common Stock'}, {'description': 'Apple Flavor & Fragrance Group Co Ltd', 'displaySymbol': '603020.SS', 'symbol': '603020.SS', 'type': 'Common Stock'}, {'description': 'Maui Land & Pineapple Company Inc', 'displaySymbol': 'MLP', 'symbol': 'MLP', 'type': 'Common Stock'}, {'description': 'Apple iSports Group Inc', 'displaySymbol': 'AAPI', 'symbol': 'AAPI', 'type': 'Common Stock'}, {'description': 'Apple International Co Ltd', 'displaySymbol': '2788.T', 'symbol': '2788.T', 'type': 'Common Stock'}, {'description': 'Applepark Co Ltd', 'displaySymbol': '164A.T', 'symbol': '164A.T', 'type': 'Common Stock'}, {'description': 'Pineapple Inc', 'displaySymbol': 'PNPL', 'symbol': 'PNPL', 'type': 'Common Stock'}, {'description': 'Pineapple Resources Bhd', 'displaySymbol': 'PINEAPP.KL'

In [161]:
first_batch = companies[:50]
second_batch = companies[50:]

In [16]:
from thefuzz import fuzz, process
import finnhub
import pandas as pd
import time  # for delays

# Finnhub client
api_key = "d3t3huhr01qqdgfu1rngd3t3huhr01qqdgfu1ro0"
finnhub_client = finnhub.Client(api_key=api_key)

final_data = []

def fetch_symbols(batch):
    for name in batch:
        query = name[:20]
        result = finnhub_client.symbol_lookup(query)
        
        if result['count'] > 0:
            candidates = result['result']
            descriptions = [c['description'] for c in candidates]

            best_match, score = process.extractOne(name, descriptions, scorer=fuzz.token_set_ratio)
            matched_item = next((c for c in candidates if c['description'] == best_match), None)

            if matched_item:
                final_data.append({
                    "Company": name,
                    "Best_Match": best_match,
                    "Symbol": matched_item['symbol'],
                    "Score": score
                })
            else:
                final_data.append({
                    "Company": name,
                    "Best_Match": None,
                    "Symbol": None,
                    "Score": None
                })
        else:
            final_data.append({
                "Company": name,
                "Best_Match": None,
                "Symbol": None,
                "Score": None
            })

# First batch
first_batch = companies[:50]
fetch_symbols(first_batch)

# Wait 60 seconds (or longer if needed) before second batch
time.sleep(60)

# Second batch
second_batch = companies[50:]
fetch_symbols(second_batch)

# Final DataFrame
df_symbol = pd.DataFrame(final_data)
print(df_symbol)


                    Company              Best_Match     Symbol  Score
0                   Walmart             Walmart Inc        WMT  100.0
1                    Amazon          Amazon.com Inc       AMZN  100.0
2        UnitedHealth Group  UnitedHealth Group Inc        UNH  100.0
3                     Apple               Apple Inc       AAPL  100.0
4                CVS Health         CVS Health Corp        CVS  100.0
..                      ...                     ...        ...    ...
95         General Dynamics   General Dynamics Corp         GD  100.0
96                Coca-Cola            Coca-Cola Co         KO  100.0
97                     TIAA      Tiaan Consumer Ltd  540108.BO   36.0
98  The Travelers Companies                    None       None    NaN
99                Eli Lilly        Eli Lilly and Co        LLY  100.0

[100 rows x 4 columns]


In [162]:
# We use fuzzy algorithm to match the correct company name and it's symbol
from thefuzz import fuzz, process
import finnhub
import pandas as pd

# Finnhub client
api_key = "d3t3huhr01qqdgfu1rngd3t3huhr01qqdgfu1ro0"
finnhub_client = finnhub.Client(api_key=api_key)

final_data = []

for name in first_batch:
    query = name[:20]
    result = finnhub_client.symbol_lookup(query)
    if result['count'] > 0:
        # Get all possible matches
        candidates = result['result']
        descriptions = [c['description'] for c in candidates]

        # Find best fuzzy match
        best_match, score = process.extractOne(name, descriptions, scorer=fuzz.token_set_ratio)

        # Get symbol for that best match
        matched_item = next((c for c in candidates if c['description'] == best_match), None)

        if matched_item:
            final_data.append({
                "Company": name,
                "Best_Match": best_match,
                "Symbol": matched_item['symbol'],
                "Score": score
            })
        else:
            final_data.append({
                "Company": name,
                "Best_Match": None,
                "Symbol": None,
                "Score": None
            })
    else:
        final_data.append({
            "Company": name,
            "Best_Match": None,
            "Symbol": None,
            "Score": None
        })

df_symbol = pd.DataFrame(final_data)
print(df_symbol)


                     Company                    Best_Match Symbol  Score
0                    Walmart                   Walmart Inc    WMT  100.0
1                     Amazon                Amazon.com Inc   AMZN  100.0
2         UnitedHealth Group        UnitedHealth Group Inc    UNH  100.0
3                      Apple                     Apple Inc   AAPL  100.0
4                 CVS Health               CVS Health Corp    CVS  100.0
5         Berkshire Hathaway        Berkshire Hathaway Inc  BRK.A  100.0
6                   Alphabet                  Alphabet Inc  GOOGL  100.0
7                 ExxonMobil                          None   None    NaN
8       McKesson Corporation                          None   None    NaN
9                    Cencora                   Cencora Inc    COR  100.0
10            JPMorgan Chase           JPMorgan Chase & Co    JPM  100.0
11                    Costco         Costco Wholesale Corp   COST  100.0
12                     Cigna                   Cign

In [163]:
# We use fuzzy algorithm to match the correct company name and it's symbol
for name in second_batch:
    query = name[:20]
    result = finnhub_client.symbol_lookup(query)
    if result['count'] > 0:
        # Get all possible matches
        candidates = result['result']
        descriptions = [c['description'] for c in candidates]

        # Find best fuzzy match
        best_match, score = process.extractOne(name, descriptions, scorer=fuzz.token_set_ratio)

        # Get symbol for that best match
        matched_item = next((c for c in candidates if c['description'] == best_match), None)

        if matched_item:
            final_data.append({
                "Company": name,
                "Best_Match": best_match,
                "Symbol": matched_item['symbol'],
                "Score": score
            })
        else:
            final_data.append({
                "Company": name,
                "Best_Match": None,
                "Symbol": None,
                "Score": None
            })

df_symbol = pd.DataFrame(final_data)



In [164]:
df_symbol

,Company,Best_Match,Symbol,Score
0,Walmart,Walmart Inc,WMT,100.0
1,Amazon,Amazon.com Inc,AMZN,100.0
2,UnitedHealth Group,UnitedHealth Group Inc,UNH,100.0
3,Apple,Apple Inc,AAPL,100.0
4,CVS Health,CVS Health Corp,CVS,100.0
...,...,...,...,...
83,Ingram Micro,Ingram Micro Holding Corp,INGM,100.0
84,General Dynamics,General Dynamics Corp,GD,100.0
85,Coca-Cola,Coca-Cola Co,KO,100.0
86,TIAA,Tiaan Consumer Ltd,540108.BO,36.0


In [165]:
df_unmatched = df_wiki[~df_wiki["Company"].isin(df_symbol["Company"])]
print("Companies without matches:", len(df_unmatched))
df_unmatched[["Company"]]


Companies without matches: 12


,Company
52,Energy Transfer Partners
53,RTX Corporation
56,Progressive Corporation
64,Merck & Co.
68,New York Life Insurance Company
70,Publix
71,Nationwide Mutual Insurance Company
86,Oracle Corporation
88,John Deere
90,Liberty Mutual


In [166]:
df_wiki

,Rank,Company,Industry,Revenue (USD millions),Revenue growth,Employees,Headquarters
0,1,Walmart,Retail,"680,985",5.1%,"2,100,000","Bentonville, Arkansas"
1,2,Amazon,Retail and cloud computing,"637,959",11.0%,"1,556,000","Seattle, Washington"
2,3,UnitedHealth Group,Healthcare,"400,278",7.7%,"400,000","Minnetonka, Minnesota"
3,4,Apple,Electronics industry,"391,035",2.0%,"164,000","Cupertino, California"
4,5,CVS Health,Healthcare,"372,809",4.2%,"259,500","Woonsocket, Rhode Island"
...,...,...,...,...,...,...,...
95,96,General Dynamics,Aerospace and defense,"47,716",12.9%,"117,000","Reston, Virginia"
96,97,Coca-Cola,Beverage,"47,061",2.9%,"69,700","Atlanta, Georgia"
97,98,TIAA,Financials,"46,946",2.6%,"15,623","New York City, New York"
98,99,The Travelers Companies,Insurance,"46,423",12.2%,"34,000","New York City, New York"


In [167]:
df_merged = df_wiki.merge(df_symbol[['Company', 'Symbol']], on='Company', how='left')





In [168]:
df_merged

,Rank,Company,Industry,Revenue (USD millions),Revenue growth,Employees,Headquarters,Symbol
0,1,Walmart,Retail,"680,985",5.1%,"2,100,000","Bentonville, Arkansas",WMT
1,2,Amazon,Retail and cloud computing,"637,959",11.0%,"1,556,000","Seattle, Washington",AMZN
2,3,UnitedHealth Group,Healthcare,"400,278",7.7%,"400,000","Minnetonka, Minnesota",UNH
3,4,Apple,Electronics industry,"391,035",2.0%,"164,000","Cupertino, California",AAPL
4,5,CVS Health,Healthcare,"372,809",4.2%,"259,500","Woonsocket, Rhode Island",CVS
...,...,...,...,...,...,...,...,...
95,96,General Dynamics,Aerospace and defense,"47,716",12.9%,"117,000","Reston, Virginia",GD
96,97,Coca-Cola,Beverage,"47,061",2.9%,"69,700","Atlanta, Georgia",KO
97,98,TIAA,Financials,"46,946",2.6%,"15,623","New York City, New York",540108.BO
98,99,The Travelers Companies,Insurance,"46,423",12.2%,"34,000","New York City, New York",NaN


In [170]:
print(df_merged.head())
print("✅ Total rows:", len(df_merged))
print("✅ Matched Symbols:", df_merged['Symbol'].notna().sum())
print("🚫 Missing symbols:", df_merged['Symbol'].isna().sum())

  Rank             Company                    Industry Revenue (USD millions)  \
0    1             Walmart                      Retail                680,985   
1    2              Amazon  Retail and cloud computing                637,959   
2    3  UnitedHealth Group                  Healthcare                400,278   
3    4               Apple        Electronics industry                391,035   
4    5          CVS Health                  Healthcare                372,809   

  Revenue growth  Employees              Headquarters Symbol  
0           5.1%  2,100,000     Bentonville, Arkansas    WMT  
1          11.0%  1,556,000       Seattle, Washington   AMZN  
2           7.7%    400,000     Minnetonka, Minnesota    UNH  
3           2.0%    164,000     Cupertino, California   AAPL  
4           4.2%    259,500  Woonsocket, Rhode Island    CVS  
✅ Total rows: 100
✅ Matched Symbols: 77
🚫 Missing symbols: 23


In [128]:
df_missing = df_merged[df_merged['Symbol'].isna()]
print("Companies with missing symbols:")
print(df_missing['Company'].tolist())


Companies with missing symbols:
['ExxonMobil', 'McKesson Corporation', 'Chevron Corporation', 'Ford Motor Company', 'The Home Depot', 'Fannie Mae', 'State Farm', 'Freddie Mac', 'Target Corporation', 'The Walt Disney Company', 'Archer Daniels Midland', 'Energy Transfer Partners', 'RTX Corporation', 'Progressive Corporation', 'Merck & Co.', 'New York Life Insurance Company', 'Publix', 'Nationwide Mutual Insurance Company', 'Oracle Corporation', 'John Deere', 'Liberty Mutual', 'USAA', 'The Travelers Companies']


In [175]:
df_merged = df_merged[df_merged['Symbol'].notna()]


In [179]:
df_merged = df_merged.reset_index(drop=True)
df_merged

,Rank,Company,Industry,Revenue (USD millions),Revenue growth,Employees,Headquarters,Symbol
0,1,Walmart,Retail,"680,985",5.1%,"2,100,000","Bentonville, Arkansas",WMT
1,2,Amazon,Retail and cloud computing,"637,959",11.0%,"1,556,000","Seattle, Washington",AMZN
2,3,UnitedHealth Group,Healthcare,"400,278",7.7%,"400,000","Minnetonka, Minnesota",UNH
3,4,Apple,Electronics industry,"391,035",2.0%,"164,000","Cupertino, California",AAPL
4,5,CVS Health,Healthcare,"372,809",4.2%,"259,500","Woonsocket, Rhode Island",CVS
...,...,...,...,...,...,...,...,...
72,95,Ingram Micro,Technology,"47,984",-0.1%,"26,125","Irvine, California",INGM
73,96,General Dynamics,Aerospace and defense,"47,716",12.9%,"117,000","Reston, Virginia",GD
74,97,Coca-Cola,Beverage,"47,061",2.9%,"69,700","Atlanta, Georgia",KO
75,98,TIAA,Financials,"46,946",2.6%,"15,623","New York City, New York",540108.BO


In [192]:
symbol_list = df_merged["Symbol"].tolist()
len(symbol)

77

In [194]:
df_merged['Symbol'] = df_merged['Symbol'].str.strip()

In [195]:
import time
# Suppose you already have df_merged with 'Symbol' column
api_key = "kT2OQrwq84sf88N7PYmDOLYb1uhbdRkZ"   # <-- replace with your real API key

data_list = []  # store results here

for symbol in df_merged['Symbol'].dropna():
    url = f"https://financialmodelingprep.com/api/v3/profile/{symbol}?apikey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        company_data = response.json()
        if company_data:
            data_list.append(company_data[0])
    else:
        print(f"Error fetching {symbol}: {response.status_code}")
    
    time.sleep(1)  # wait 1 second between requests


Error fetching WMT: 403
Error fetching AMZN: 403
Error fetching UNH: 403
Error fetching AAPL: 403
Error fetching CVS: 403


KeyboardInterrupt: 

In [193]:
import requests
import pandas as pd

# Suppose you already have df_merged with 'Symbol' column
api_key = "kT2OQrwq84sf88N7PYmDOLYb1uhbdRkZ"   # <-- replace with your real API key

data_list = []  # store results here

for symbol in df_merged['Symbol'].dropna():   # skip empty symbols
    url = f"https://financialmodelingprep.com/api/v3/profile/{symbol}?apikey={api_key}"
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            company_data = response.json()
            if company_data:  # not empty
                data_list.append(company_data[0])  # take the first element
        else:
            print(f"Error fetching {symbol}: {response.status_code}")
    except Exception as e:
        print(f"Failed {symbol}: {e}")

# Convert list of dicts → DataFrame
df_api = pd.DataFrame(data_list)

print(df_api.head())
print(f"Fetched {len(df_api)} companies successfully.")


Error fetching WMT: 403
Error fetching AMZN: 403
Error fetching UNH: 403
Error fetching AAPL: 403
Error fetching CVS: 403
Error fetching BRK.A: 403
Error fetching GOOGL: 403
Error fetching COR: 403
Error fetching JPM: 403
Error fetching COST: 403
Error fetching CI: 403
Error fetching MSFT: 403
Error fetching CAH: 403
Error fetching BAC: 403
Error fetching GM: 403
Error fetching ELV: 403
Error fetching C: 403
Error fetching META: 403
Error fetching CNC: 403
Error fetching WBA: 403
Error fetching KR: 403
Error fetching PSX: 403
Error fetching MPC: 403
Error fetching VZ: 403
Error fetching NVDA: 403
Error fetching GS: 403
Error fetching WFC: 403
Error fetching VLO: 403
Error fetching CCZ: 403
Error fetching T: 403
Error fetching HUM: 403
Error fetching MS: 403
Error fetching SNEX: 403
Error fetching TSLA: 403
Error fetching DELL: 403
Error fetching PEP: 403
Error fetching UPS: 403
Error fetching JNJ: 403
Error fetching FDX: 403


KeyboardInterrupt: 

In [207]:
#all_data = []

api_key = "kT2OQrwq84sf88N7PYmDOLYb1uhbdRkZ"  # replace with your real API key later 
company = "APPL"
url1=f"https://financialmodelingprep.com/stable/profile?symbol=AAPL&apikey=kT2OQrwq84sf88N7PYmDOLYb1uhbdRkZ"
url =f"https://financialmodelingprep.com/stable/profile?symbol={company}&apikey={api_key}"

r = requests.get(url1)
print(r.json())



[{'symbol': 'AAPL', 'price': 260.385, 'marketCap': 3864214950150, 'beta': 1.094, 'lastDividend': 1.02, 'range': '169.21-265.29', 'change': 1.935, 'changePercentage': 0.74869, 'volume': 11915338, 'averageVolume': 54967137, 'companyName': 'Apple Inc.', 'currency': 'USD', 'cik': '0000320193', 'isin': 'US0378331005', 'cusip': '037833100', 'exchangeFullName': 'NASDAQ Global Select', 'exchange': 'NASDAQ', 'industry': 'Consumer Electronics', 'website': 'https://www.apple.com', 'description': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and do

In [213]:
import requests
import pandas as pd
import time

api_key = "kT2OQrwq84sf88N7PYmDOLYb1uhbdRkZ"

# Suppose df_wiki has a column 'Symbol' with 88 symbols
#symbols = df_wiki['Symbol'].tolist()
symbols = df_merged['Symbol'].tolist()
profiles = []

for company in symbols:
    url = f"https://financialmodelingprep.com/stable/profile?symbol={company}&apikey={api_key}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        if data:  # make sure list is not empty
            profiles.append(data[0])  # data is returned as a list with 1 dict
    else:
        print(f"⚠️ Failed for {company}: {response.status_code}")
    
    # optional: small delay to be polite
    time.sleep(0.1)

# Convert to DataFrame
df_profiles = pd.DataFrame(profiles)

# Check result
print(df_profiles.head())



  symbol     price     marketCap   beta  lastDividend          range  change  \
0    WMT  106.0000  8.451222e+11  0.671        0.9125   79.81-109.58 -1.1400   
1   AMZN  220.1320  2.347688e+12  1.281        0.0000  161.38-242.52  2.1820   
2    UNH  358.7265  3.248891e+11  0.478        8.6200   234.6-630.73 -2.7635   
3   AAPL  260.0712  3.859558e+12  1.094        1.0200  169.21-265.29  1.6212   
4    CVS   81.0000  1.027344e+11  0.609        1.9950    43.56-83.52 -1.0100   

   changePercentage    volume  averageVolume  ...          city state  \
0          -1.06403   3767074     16517159.0  ...   Bentonville    AR   
1           1.00115  13437960     44254714.0  ...       Seattle    WA   
2          -0.76447   2930117     15174048.0  ...  Eden Prairie    MN   
3           0.62728  12114580     54967137.0  ...     Cupertino    CA   
4          -1.23156   2534352      7853815.1  ...    Woonsocket    RI   

          zip                                              image     ipoDate  \


In [1]:
df_profiles

NameError: name 'df_profiles' is not defined